In [10]:
import numpy as np
from scipy.integrate import quad
from scipy.special import gamma


In [2]:
filename = "scenario_1.npz"

In [3]:
data = np.load(filename)
S = data["S"]
I = data["I"]
R = data["R"]

S_0 = S[0]
I_0 = I[0]
R_0 = R[0]
N_0 = S_0 + I_0 + R_0
T = len(S)
p = 0.01

print(S_0, I_0, R_0, N_0, T)

999950.0 50.0 0.0 1000000.0 100


In [4]:
# define the prior distributions


def prior_delta(T, p):
    # maybe the first element has to be 1
    return np.concatenate([[1], np.random.binomial(1, p, size=T - 1)])


def prior_r(K: int):
    return np.random.gamma(shape=0.1, scale=0.1, size=K)


def prior_b(K):
    return np.random.gamma(shape=0.1, scale=0.1, size=K)


def prior_gamma(delta, r):
    eta = np.cumsum(delta)
    gamma = np.random.beta(a=r[eta - 1], b=1)
    return gamma


def prior_beta(delta, b):
    eta = np.cumsum(delta)
    beta = np.random.exponential(scale=b[eta - 1])
    return beta

In [5]:
# define the sampling parameters
n_samplings = 2

delta = np.zeros((n_samplings, T))
r = np.zeros((n_samplings, T))
b = np.zeros((n_samplings, T))
beta = np.zeros((n_samplings, T))
gamma = np.zeros((n_samplings, T))

In [6]:
def init(T, p):
    delta_tmp = prior_delta(T, p)
    K = np.sum(delta_tmp)
    r_temp = prior_r(K)
    b_temp = prior_b(K)

    delta[0, :] = delta_tmp
    r[0, 0:K] = r_temp
    b[0, 0:K] = b_temp
    gamma[0, :] = prior_gamma(delta_tmp, r_temp)
    beta[0, :] = prior_beta(delta_tmp, b_temp)
    

In [9]:
init(T, p)
# print(delta[0])
# print(beta[0])

In [ ]:
def add(delta_):
    zero_index = np.random.shuffle(np.where(delta_ == 0)[0])[0]
    delta_[zero_index] = 1
    return delta_


def delete(delta_):
    zero_index = np.random.shuffle(np.where(delta_ == 1)[0])[0]
    delta_[zero_index] = 0
    return delta_


def swap(delta_):
    tmp = delta_[1:] + delta_[:-1]
    shuffling_index = np.random.shuffle(np.where(tmp == 1)[0] + 1)[0]

    tmp_value = delta_[shuffling_index - 1]
    delta_[shuffling_index - 1] = delta_[shuffling_index]
    delta_[shuffling_index] = tmp_value

    return delta_

In [ ]:
for n in range(1, n_samplings):
    delta_g = delta[n - 1, :]
    K_g = np.sum(delta_g)
    delta_ = delta_g
    if K_g == 1:  # add
        delta_ = add(delta_)
    elif K_g == T:  # delete
        delta_ = delete(delta_)
    else:  # random
        case = np.random.random()
        if case < 1 / 3:
            delta_ = add(delta_)
        elif case < 2 / 3:
            delta_ = delete(delta_)
        else:
            delta_ = swap(delta_)
    K_ = np.sum(delta_)
    mh_delta = (p / (1 - p)) ** (K_ - K_g)

In [ ]:
def likelihood(beta, gamma, delta, K):

    eta = np.cumsum(delta)
    fac_1 = 1
    for k in range(1, K):
        t_indexes = np.where(eta == k)[0]

In [ ]:
def integrand_1(x, beta_):
    return (
        0.1**0.1 * x ** (-0.9) * np.exp(-0.1 * x) / gamma(0.1) * x * np.exp(-beta_ * x)
    )